In [64]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
sc.stop()

# Create a local StreamingContext with two working thread and batch interval of 1 second
sc = SparkContext("local[2]", "SparkStreaming Exercise")
ssc = StreamingContext(sc, 10)
ssc.checkpoint("/Users/pravinkumar/Documents/Spark/Results/checkPointForStreaming")

In [65]:
# Create a DStream that will connect to hostname:port, like localhost:9999
socket = ssc.socketTextStream("localhost", 9999)

In [66]:
# Joining two files 
deckDstream = socket.map(lambda rec: rec.split("|")).map(lambda rec: (rec[0], rec[1]))
deckRDD = sc.textFile("/Users/pravinkumar/Documents/Spark/testData/cards/deckofcards.txt").map(lambda rec: rec.split('|')).map(lambda rec: (rec[0], rec[2]))

deckJoin = deckDstream.transform(lambda rdd: rdd.join(deckRDD))

def prnt(x):
    x.pprint()

# Applying the function prnt to each RDD in a DStream
deckJoin.pprint()

In [67]:
# Count each color in a deck in each batch using reduceByKey
deckDstrmColor = deckDstream.map(lambda rec: (rec[0], 1))
wordCounts = deckDstrmColor.reduceByKey(lambda x, y: x + y)

# Print the first ten elements of each RDD generated in this DStream to the console
wordCounts.pprint()

In [68]:
# Count each color in a deck in each batch using countByValue
# countByValue When called on a DStream of elements of type K, return a new DStream of (K, Long) pairs 
# where the value of each key is its frequency in each RDD of the source DStream.

wordCounts = deckDstrmColor.countByValue()

# Print the first ten elements of each RDD generated in this DStream to the console
wordCounts.pprint()

In [69]:
# Each word count in a DStream using flatMapValue in RDD Transformation. just a different way of trying.

wordCounts = deckDstream.map(lambda rec: (1, rec)).transform(lambda rdd: rdd.flatMapValues(lambda rec: rec)).map(lambda rec: (rec[1], rec[0])).reduceByKey(lambda Acc, value: acc + value)

# Print the first ten elements of each RDD generated in this DStream to the console
wordCounts.pprint()

In [70]:
# updateStateByKey is used here as like a static storage where all the words with its count is stored 
# and will be incremanted on getting the words and displaying it


def updateFunction(newValues, runningCount):
    if runningCount is None:
        runningCount = 0
    return sum(newValues, runningCount)

wordCounts = socket.flatMap(lambda rec: rec.split("|")).map(lambda rec: (rec, 1)).updateStateByKey(updateFunction)

# Print the first ten elements of each RDD generated in this DStream to the console
wordCounts.pprint()

In [ ]:
ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate

-------------------------------------------
Time: 2017-07-08 23:26:30
-------------------------------------------

-------------------------------------------
Time: 2017-07-08 23:26:40
-------------------------------------------

-------------------------------------------
Time: 2017-07-08 23:26:50
-------------------------------------------

-------------------------------------------
Time: 2017-07-08 23:27:00
-------------------------------------------

-------------------------------------------
Time: 2017-07-08 23:27:10
-------------------------------------------

-------------------------------------------
Time: 2017-07-08 23:27:20
-------------------------------------------

-------------------------------------------
Time: 2017-07-08 23:27:30
-------------------------------------------

-------------------------------------------
Time: 2017-07-08 23:27:40
-------------------------------------------

-------------------------------------------
Time: 2017-07-08 23:27:50
----------

In [19]:
ssc.stop()